In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T


### `Step 1`: Quick Analysis

In [0]:
df_sales_customers = spark.read.table("samples.bakehouse.sales_customers")
df_sales_customers.printSchema()
df_sales_customers.limit(3).toPandas()

In [0]:
# distinct values per each column: 300 rows
df_sales_customers.select([F.countDistinct(F.col(c)).alias(c) for c in df_sales_customers.columns]).toPandas()

# count NULLs per each column: no NULLs, this is nice
df_sales_customers.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_sales_customers.columns]).toPandas()

### `Step 2.1`: Data Quality Issue - `gender`

In `01_bakehouse_exloratory_analysis.ipynb` I noticed 'Matthew' can be female.

Let's create a view and quickly see if this is a data quality issue or not 


In [0]:
df_sales_customers.createOrReplaceTempView("vw_sales_customers")


In [0]:
%%sql
WITH CTE_DISTINCT_first_name_gender AS (
    SELECT DISTINCT
         first_name
        ,gender
    FROM vw_sales_customers
)

,CTE_first_name_with_both_genders AS (
    SELECT 
         first_name
        ,COUNT(1) AS count_first_name
    FROM CTE_DISTINCT_first_name_gender
    GROUP BY 
        first_name
    HAVING 
        COUNT(1) > 1
)

SELECT DISTINCT 
     vw.first_name
    ,vw.gender
    ,cte.count_first_name
FROM vw_sales_customers vw
INNER JOIN CTE_first_name_with_both_genders cte -- to filter only on duping names with both genders
    ON vw.first_name = cte.first_name 
ORDER BY 
     cte.count_first_name DESC
    ,vw.first_name
    ,vw.gender


There ☝️ are `74/2 == 37` names with both genders.
Some of them are indeex unisex, but not all.
This means we need to do some extra work to get the gender right.

Let's import open-source baby-names data from  official website of the US government: https://www.ssa.gov/oact/babynames/limits.html

In [0]:
babyNames_file_path = "dbfs:/Volumes/workspace/default/testvolume/US_SSA_Baby_Names_2024.txt"

# txt file has no header, thus I will create a custom schema for the dataframe
# plus, having a predefined schema saves a lot of time for Spark to infer the schema
babyNamesSchema = T.StructType([
    T.StructField("name",   T.StringType(), True),
    T.StructField("gender", T.StringType(), True),
    T.StructField("count",  T.IntegerType(), True)
])

df_babyNames = (
    spark.read.csv(
        babyNames_file_path,
        sep = ",",
        schema = babyNamesSchema
    ) 
    .select("name", "gender") 
    .withColumnRenamed("name", "first_name") 
    .withColumn("gender", F.when(F.col("gender") == "F", "female").otherwise("male")) 
    .withColumnRenamed("gender", "gender_correct")
    .dropDuplicates(["first_name", "gender_correct"])
)

df_babyNames.where("first_name in ('Asyah', 'Atari', 'Owen')").orderBy("first_name", "gender_correct").show()

Some names (like above☝️) can be both male and female. 

Let's split `df_babyNames` into two dfs: w/ unisex names and w/ single-gender names

In [0]:
# single gender names
df_babyNames_single_gender = (
    df_babyNames
        .groupBy("first_name")
        .count()
        .where(F.col("count") == 1)
        .drop("count")
)

# `df_babyNames`: create a list of non-duping names (aka single gender)
babyNames_single_gender_names_list = [
    row["first_name"] 
    for row in df_babyNames_single_gender.select("first_name").collect()
]

# let's split `df_babyNames` into two dfs: one w/ unisex names and the other w/ single-gender names
df_babyNames_unisex_names = (
    df_babyNames
    .where(~F.col("first_name").isin(babyNames_single_gender_names_list))
)

df_babyNames_single_gender_names = (
    df_babyNames
    .where(F.col("first_name").isin(babyNames_single_gender_names_list))
)

# quick check on count
print(
    f"Total:{df_babyNames.count()}, " #31904
    f"Duping names:{df_babyNames_unisex_names.count()}, " #5358
    f"Non-duping names:{df_babyNames_single_gender_names.count()}" #26546
)


Let's do the same with `df_sales_customers`: split into duping and non-dupling names dfs

In [0]:
# `df_sales_customers`: create a list of duping names
sales_customers_duping_names =  (
    df_sales_customers
        .groupBy("first_name")
        .count()
        .where(F.col("count") > 1)
        .drop("count")
)

sales_customers_duping_names_list = [
    row["first_name"] 
    for row in sales_customers_duping_names.select("first_name").collect()
]

# let's split `df_sales_customers` into two dfs: one w/ duping names and the other w/ non-duping names
df_sales_customers_duping_names = (
    df_sales_customers
    .where(F.col("first_name").isin(sales_customers_duping_names_list))
)

df_sales_customers_nonDuping_names = (
    df_sales_customers
    .where(~F.col("first_name").isin(sales_customers_duping_names_list))
)

# quick check on count
print(
    f"Total:{df_sales_customers.count()}, " #300
    f"Duping names:{df_sales_customers_duping_names.count()}, " #180
    f"Non-duping names:{df_sales_customers_nonDuping_names.count()}" #120
)

In [0]:
df_babyNames_single_gender_names.show(3)

We need to fix only `df_sales_customers_duping_names`.

Let's join w/ `df_babyNames_single_gender_names` to see if there are any duping names with only one gender

In [0]:

df_sales_customers_duping_names_corrected = (
    df_sales_customers_duping_names
        .join(df_babyNames_single_gender_names, on = "first_name", how = "left")
        .withColumn("gender_final", F.coalesce(
            F.col("gender_correct"), #1st - `gender_correct` from `df_babyNames_single_gender_names`
            F.col("gender")          #2nd - `gender` from `df_sales_customers_duping_names`
            ))
        .drop("gender_correct", "gender") #dropping original gender fields
        .withColumnRenamed("gender_final", "gender")     
)

df_sales_customers_duping_names_corrected.limit(3).toPandas()

Now that we have `df_sales_customers_duping_names_corrected` and `df_sales_customers_nonDuping_names`, we can combine them into the one table

In [0]:
df_sales_customers_correctGender = (
    df_sales_customers_nonDuping_names
        #using `unionByName` to avoid field disordering issue
        .unionByName(df_sales_customers_duping_names_corrected) 
)

# quick check on count
print(
    f"fixed:{df_sales_customers_correctGender.count()}, " #300
    f"Non-dupling names:{df_sales_customers_nonDuping_names.count()}, " #120
    f"Duping names:{df_sales_customers_duping_names.count()}" #180
)

### `Step 2.2`: Data Quality Issue - `country` & `state`

Also, in `01_bakehouse_exloratory_analysis.ipynb` I noticed cities are improperly mapped to countries.

Some examples:

In [0]:
(
    df_sales_customers_correctGender
        .where("state IN ('Nevada', 'Kansas', 'Washington')")
        .select("city", "state", "country")
        .distinct()
        .orderBy("country", "state")
        .show()
)

Another spotted issue: states are inconsistet, can be full ('Alabama') and abbreviation ('AL')

In [0]:
df_sales_customers_correctGender.select("state").limit(10).orderBy("state").toPandas()

This means, we need need to do some extra work to:
1. get city-country mapping right
2. standarzie state names

Let's import open-source data from `World Cities Database`:
* https://simplemaps.com/data/world-cities
* https://simplemaps.com/data/us-cities

In [0]:
worldCities_file_path = "dbfs:/Volumes/workspace/default/testvolume/worldcities.csv"

df_worldCities = (
    spark.read.csv(
        worldCities_file_path,
        sep = ",",
        header=True
    ) 
    .select("city", "country") 
    .withColumnRenamed("country", "country_correct") 
    .withColumn("country_correct",
                 F.when(F.col("country_correct") == "United States", "USA")
                 .otherwise(F.col("country_correct"))) 
    .dropDuplicates(["city", "country_correct"])
)

df_worldCities.limit(3).show()

In [0]:
usCities_file_path = "/Volumes/workspace/default/testvolume/uscities.csv"
                           #/Volumes/workspace/default/testvolume/us-cities-demographics.csv

df_usStates = (
        spark.read.csv(
        usCities_file_path,
        sep = ",",
        header=True
    )
    .select("state_name", "state_id")
    .distinct()
    .withColumn("us_country_correct", F.lit("USA"))
    .withColumn("us_coninent_correct", F.lit("North America"))
    .withColumnRenamed("state_name", "state_correct")
    .withColumnRenamed("state_id", "state_abbreviation")
    .dropDuplicates(["state_correct", "state_abbreviation", "us_country_correct", "us_coninent_correct"])
)
df_usStates.limit(10).toPandas()
#df_usCities.limit(10).show()

Let's join w/ `df_usStates` & `df_worldCities` to get the correct country and fix states

In [0]:
df_sales_customers_correctGender_correctCountry = (
    df_sales_customers_correctGender
        .join(
            df_usStates,
        # because `df_sales_customers_correctGender.state` contains both full and short state names, we need to join on both
            (df_sales_customers_correctGender.state == df_usStates.state_correct) | 
            (df_sales_customers_correctGender.state == df_usStates.state_abbreviation),
            "left")
        .join(df_worldCities, on="city", how="left")
        .withColumn("country", F.coalesce(
            F.col("us_country_correct"), # if `us_country_correct` is not null, use it ('USA' only)
            F.col("country_correct"), # use `country_correct` but only after us_country_correct
            F.col("country")
            ))
        .withColumn("continent", F.coalesce(
            F.col("us_coninent_correct"), # if not null, use it ('North America' only)
            F.col("continent")
            ))
        .withColumn("state", F.coalesce(
            F.col("state_correct"), #if not null, use `state_correct` from `df_usStates`
            F.col("state") #otherwise, take `state` from `df_sales_customers_correctGender`
            ))
        .drop("us_country_correct", "country_correct", "us_coninent_correct", "state_correct", "state_abbreviation")
)

#df_sales_customers_correctGender_correctCountry.where("gender != gender_correct").show(3)
df_sales_customers_correctGender_correctCountry.where('customerID = 2000269').limit(10).toPandas()#show()

### `Step 3`: Last Transfromations

In [0]:
# Selecting Relevant Fields
selectedColumns = [
        "customerID", "first_name", "last_name", "email_address",
        "gender", "city", "state", "country", "continent"
]

df_sales_customers_selectedColumns = df_sales_customers_correctGender_correctCountry.select(selectedColumns)

In [0]:
# distinct values per each column: 300 rows
df_sales_customers_selectedColumns.select([F.countDistinct(F.col(c)).alias(c) for c in df_sales_customers_selectedColumns.columns]).toPandas()

In [0]:
df_sales_customers_selectedColumns.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_sales_customers_selectedColumns.columns]).toPandas()

In [0]:
#capitalize
df_sales_customers_selectedColumns_capitalized = (
    df_sales_customers_selectedColumns
        .withColumn("first_name", F.initcap(F.trim("first_name"))) 
        .withColumn("last_name", F.initcap(F.trim("last_name"))) 
        .withColumn("city", F.initcap(F.trim("city"))) 
        .withColumn("country",
                    F.when(F.col("country") == 'USA', F.lit("USA"))
                    .otherwise(F.initcap(F.trim("country")))) 
        .withColumn("gender", F.initcap(F.trim("gender")))
)

In [0]:
df_sales_customers_selectedColumns_capitalized.limit(10).toPandas() 

In [0]:
df_sales_customers_selectedColumns_capitalized = (
    df_sales_customers_selectedColumns_capitalized
        .withColumn("customer_key", F.monotonically_increasing_id())
        .select("cusotmer_key",
                "customerID",
                "first_name",
                "last_name",
                "")
)

In [0]:

df_sales_customers_correctGender_correctCountry.select('state').distinct().orderBy('state').show()